# 装饰器

> 装饰器本质上是一个Python函数，它可以让其他函数在不需要做任何代码变动的前提下增加额外功能，装饰器的返回值也是一个函数对象。它经常用于有切面需求的场景，比如：插入日志、性能测试、事务处理、缓存、权限校验等场景。装饰器是解决这类问题的绝佳设计，有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码并继续重用<sup>[Re](https://www.cnblogs.com/cicaday/p/python-decorator.html)</sup>。  
概括的讲，装饰器的作用就是为已经存在的函数或对象添加额外的功能。

## 简单装饰器

一个装饰器的示例如下：  
其中： @ 符号就是装饰器的语法糖，它放在函数开始定义的地方，这样就可以省略最后一步再次赋值的操作。

In [4]:
def debug(func):
    def wrapper():
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func()
    return wrapper

@debug
def say_hello():
    print( "hello!")
    
say_hello()

[DEBUG]: enter say_hello()
hello!


## 含参装饰器

这是最简单的装饰器，但是有一个问题，如果被装饰的函数需要传入参数，那么这个装饰器就坏了。因为返回的函数并不能接受参数，你可以指定装饰器函数wrapper接受和原函数一样的参数，比如：

In [8]:
def debug(func):
    def wrapper(something):  # 指定一毛一样的参数
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func(something)
    return wrapper  # 返回包装过函数

@debug
def say(something):
    print( "hello {}!".format(something))
    
say('xm')

[DEBUG]: enter say()
hello xm!


这样你就解决了一个问题，但又多了N个问题。因为函数有千千万，你只管你自己的函数，别人的函数参数是什么样子，鬼知道？还好Python提供了可变参数\*args和关键字参数\*\*kwargs，有了这两个参数，装饰器就可以用于任意目标函数了。

In [12]:
def debug(func):
    def wrapper(*args, **kwargs):  # 指定宇宙无敌参数
        print( "[DEBUG]: enter {}()".format(func.__name__))
        print( 'Prepare and say...',)
        return func(*args, **kwargs)
    return wrapper  # 返回

@debug
def say(something):
    print("hello {}!".format(something))
    
say('xm')

[DEBUG]: enter say()
Prepare and say...
hello xm!


例如我们要计算几个函数的运行时间，我们可以这样写：

In [25]:
import time

# 定义装饰器
def time_calc(func):
    def wrapper(*args, **kargs):        
        start_time = time.time()        
        f = func(*args,**kargs)        
        exec_time = time.time() - start_time 
        print('{}运行时间为{}'.format(func.__name__,exec_time))
        return f    
    return wrapper   
    
# 使用装饰器
@time_calc    
def add(a, b):
    for i in range (0,5):
        time.sleep(1)
    return a+b
    
@time_calc
def sub(a, b):    
    for i in range (0,3):
        time.sleep(1)
    return a-b

print(add(1,2))
print(sub(1,2))

add运行时间为5.005070924758911
3
sub运行时间为3.000138998031616
-1


带参数的装饰器和类装饰器属于进阶的内容。在理解这些装饰器之前，最好对函数的闭包和装饰器的接口约定有一定了解。([参见Python的闭包](https://www.cnblogs.com/cicaday/p/python-closure.html))  
假设我们前文的装饰器需要完成的功能不仅仅是能在进入某个函数后打出log信息，而且还需指定log的级别，那么装饰器就会是这样的。

In [27]:
def logging(level):
    def wrapper(func):
        def inner_wrapper(*args, **kwargs):
            print("[{level}]: enter function {func}()".format(
                level=level,
                func=func.__name__))
            return func(*args, **kwargs)
        return inner_wrapper
    return wrapper

@logging(level='INFO')
def say(something):
    print( "say {}!".format(something))

# 如果没有使用@语法，等同于
# say = logging(level='INFO')(say)

@logging(level='DEBUG')
def do(something):
    print( "do {}...".format(something))

if __name__ == '__main__':
    say('hello')
    do("my work")

[INFO]: enter function say()
say hello!
[DEBUG]: enter function do()
do my work...


## 基于类实现的装饰器

装饰器函数其实是这样一个接口约束，它必须接受一个callable对象作为参数，然后返回一个callable对象。在Python中一般callable对象都是函数，但也有例外。只要某个对象重载了__call__()方法，那么这个对象就是callable的。  
回到装饰器上的概念上来，装饰器要求接受一个callable对象，并返回一个callable对象（不太严谨，详见后文）。那么用类来实现也是也可以的。我们可以让类的构造函数__init__()接受一个函数，然后重载__call__()并返回一个函数，也可以达到装饰器函数的效果。

In [30]:
class logging(object):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print( "[DEBUG]: enter function {func}()".format(
            func=self.func.__name__))
        return self.func(*args, **kwargs)
@logging
def say(something):
    print( "say {}!".format(something))

say('xm')

[DEBUG]: enter function say()
say xm!


如果需要通过类形式实现带参数的装饰器，那么会比前面的例子稍微复杂一点。那么在构造函数里接受的就不是一个函数，而是传入的参数。通过类把这些参数保存起来。然后在重载__call__方法是就需要接受一个函数并返回一个函数。

In [33]:
class logging(object):
    def __init__(self, level='INFO'):
        self.level = level
        
    def __call__(self, func): # 接受函数
        def wrapper(*args, **kwargs):
            print( "[{level}]: enter function {func}()".format(
                level=self.level,
                func=func.__name__))
            func(*args, **kwargs)
        return wrapper  #返回函数

@logging(level='INFO')
def say(something):
    print( "say {}!".format(something))
say('xm')

[INFO]: enter function say()
say xm!


# 内置装饰器
内置的装饰器和普通的装饰器原理是一样的，只不过返回的不是函数，而是类对象，所以更难理解一些。

## @property

 @property 可以把一个实例方法变成其同名属性，以支持实例访问，它返回的是一个property属性；

In [41]:
import math
class Circle:
    def __init__(self,radius): #圆的半径radius
        self.radius=radius

    @property
    def area(self):
        return math.pi * self.radius**2 #计算面积

    @property
    def perimeter(self):
        return 2*math.pi*self.radius #计算周长

# 我们可以通过实例访问到类中属性
circle=Circle(10)
print(circle.radius)

# 通过@property装饰后的方法也可以像访问数据属性一样去访问area,会触发一个函数的执行,动态计算出一个值；
print(circle.area) 
print(circle.perimeter) 

10
314.1592653589793
62.83185307179586


一个property对象还具有setter、deleter 可用作装饰器；setter是设置属性值。deleter用于删除属性值。而官方文档中给出了getter用于获取属性信息，但是实际使用中可以直接通过property获取属性信息；

In [45]:
class C:
   def __init__(self):
        self._x = None

    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

#实例化类
c = C()
# 为属性进行赋值
c.x=100
# 输出属性值
print(c.x)
# 删除属性
del c.x

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

总结：使用property：在设置属性时，可以对值对进检查，设置发生时，可以 修改设置的值，获取属性时，可以动态地计算值。

##  @classmethod

@classmethod   修饰的方法不需要实例化，不需要 self 参数，但第一个参数需要是表示自身类的 cls 参数，可以来调用类的属性，类的方法，实例化对象等。

In [46]:
class A():
    number = 10

    @classmethod
    def get_a(cls):     #cls 接收的是当前类，类在使用时会将自身传入到类方法的第一个参数
        print('这是类本身：',cls)# 如果子类调用，则传入的是子类
        print('这是类属性:',cls.number)

class B(A):
    number = 20
    pass

# 调用类方法 不需要实例化可以执行调用类方法
A.get_a()
B.get_a()

这是类本身： <class '__main__.A'>
这是类属性: 10
这是类本身： <class '__main__.B'>
这是类属性: 20


## @staticmethod
改变一个方法为静态方法，静态方法不需要传递隐性的第一参数，静态方法的本质类型就是一个函数 一个静态方法可以直接通过类进行调用，也可以通过实例进行调用；

In [48]:
import time
class Date:
    def __init__(self,year,month,day):
        self.year=year
        self.month=month
        self.day=day

    @staticmethod
    def now(): #用Date.now()的形式去产生实例,该实例用的是当前时间
        t=time.localtime() #获取结构化的时间格式
        return Date(t.tm_year,t.tm_mon,t.tm_mday) #新建实例并且返回


    @staticmethod
    def tomorrow():#用Date.tomorrow()的形式去产生实例,该实例用的是明天的时间
        t=time.localtime(time.time()+86400)
        return Date(t.tm_year,t.tm_mon,t.tm_mday)

a=Date('1987',11,27) #自己定义时间
print(a.year,a.month,a.day)
b=Date.now() #采用当前时间
print(b.year,b.month,b.day)
c=Date.tomorrow() #采用明天的时间
print(c.year,c.month,c.day)

1987 11 27
2020 7 9
2020 7 10


# functools.wraps(func)的使用

当我们使用装饰器时，我们会发现以下问题：

In [56]:
def debug(func):
    def wrapper():
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func()
    return wrapper

@debug
def say_hello():
    print( "hello!")

def say_hello1():
    print( "hello!")
wrapper
print(say_hello().__name__)  # 输出 wrapper
print(say_hello1().__name__) # 输出 say_hello

装饰器改变了原函数的名称，是我们丢失了函数原来的信息，那该怎么办呢，只需要在第一级中加入functools.wraps即可，如下所示：

In [58]:
def debug(func):
    @functools.wraps(func)
    def wrapper():
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func()
    return wrapper

@debug
def say_hello():
    print( "hello!")

def say_hello1():
    print( "hello!")
wrapper
print(say_hello().__name__)  # 输出 wrapper
print(say_hello1().__name__) # 输出 say_hello

# 多个装饰器运行顺序
一个函数还可以同时定义多个装饰器，它的执行顺序是从里到外，最先调用最里层的装饰器，最后调用最外层的装饰器。
```python
@a
@b
@c
def f ():
    pass
```